In [3]:
import requests
from bs4 import BeautifulSoup

In [25]:
url = "https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/"

In [5]:
req = requests.get(url)

In [9]:
soup = BeautifulSoup(req.content, "html.parser")

In [18]:
total_pages = int(soup.find(id='total_pages').text.strip())
total_pages

2

In [20]:
range(total_pages)

range(0, 2)

In [28]:
for page in range(total_pages):
    pg_url = url
    pg_url += f"page-{page+1}/"
    # print(pg_url)
    req_pg = requests.get(pg_url)
    soup = BeautifulSoup(req_pg.content, "html.parser")
    

https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/page-1/
https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/page-2/


In [130]:
pg_url = "https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/page-2/"
req_pg = requests.get(pg_url)
soup = BeautifulSoup(req_pg.content, "html.parser")

In [131]:
internships = soup.find_all(class_ = 'heading_4_5 profile')

In [132]:
internships[2].find('a', href=True)["href"]

'/internship/detail/artificial-intelligence-ai-python-development-work-from-home-job-internship-at-scure-infosec1655700102'

In [165]:
sub_url = 'https://internshala.com' + internships[2].find('a', href=True)["href"]
sub_url = "https://internshala.com/internship/detail/market-research-analytics-internship-in-multiple-locations-at-justdial-limited1657168942"

In [166]:
req_sub_pg = requests.get(sub_url)
sub_soup = BeautifulSoup(req_sub_pg.content, "html.parser")

In [167]:
sub_soup.find(class_ = 'profile_on_detail_page').text.strip()

'Market Research (Analytics)'

In [168]:
sub_soup.find(class_ = 'heading_6 company_name').find('a').text.strip()

'Justdial Limited'

In [169]:
[i.text.strip() for i in sub_soup.find_all(class_ = 'location_link')]

['Delhi', 'Bangalore', 'Mumbai']

In [170]:
info = sub_soup.find(class_ = 'internship_other_details_container')
other_details = info.find_all(class_ = 'item_body')

In [171]:
other_details[1].text.strip()

'2 Months'

In [172]:
other_details[2].text.strip()

'2000 /month +  Incentives'

In [173]:
other_details[3].text.strip() # apply by

"21 Jul' 22"

In [174]:
sub_soup.find(class_ = 'applications_message').text.strip()

'132 applicants'

In [163]:
skills_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Skill(s) required')

In [164]:
skills_raw = skills_raw.findNext(class_ = 'round_tabs_container')

AttributeError: 'NoneType' object has no attribute 'findNext'

In [58]:
skills_raw.find_all(class_ = 'round_tabs')

[<span class="round_tabs">Financial literacy</span>,
 <span class="round_tabs">Labour laws</span>,
 <span class="round_tabs">MS-Excel</span>,
 <span class="round_tabs">MS-PowerPoint</span>]

In [141]:
[i.text.strip() for i in skills_raw.find_all(class_ = 'round_tabs')]

['Financial literacy', 'Labour laws', 'MS-Excel', 'MS-PowerPoint']

In [61]:
perks_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Perks')
perks_raw = perks_raw.findNext(class_ = 'round_tabs_container')
[i.text.strip() for i in perks_raw.find_all(class_ = 'round_tabs')]

['Certificate', 'Letter of recommendation', 'Job offer']

In [63]:
sub_soup.find_all(class_='text-container')[-1].text.strip()

'4'

In [64]:
url

'https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/'

In [86]:
def transform_categories(category_list):
    final_params = []
    for category in category_list:
        category = category.strip().lower()
        category = category.replace(' ','-')
        final_params.append(category)
    final_params = ','.join(final_params)
    final_params+='-internship'
    return final_params

In [87]:
transform_categories(['Data Science', 'Artificial Intelligence (AI)'])
# 

'data-science,artificial-intelligence-(ai)-internship'

In [75]:
list(map(str,input("Enter different categories separated by commas* (Required)\n").split(',')))

['Data Science', ' Machine Learning']

In [96]:
def transform_locations(location_list):
    final_params = []
    for location in location_list:
        location = location.strip().lower()
        location = location.replace(' ','-')
        final_params.append(location)
    final_params = ','.join(final_params)
    final_params ='-in-'+final_params

    return final_params

In [97]:
transform_locations(["Delhi", "Mumbai"])

'-in-delhi,mumbai'

In [98]:
base_url = "https://internshala.com/internships/"

In [142]:
input = {
    "category": ["Machine Learning"],
    "wfh": "yes",
    "location": []
}

In [143]:
import pandas as pd
df = pd.DataFrame()

In [178]:
df = pd.concat([df, pd.DataFrame([input])], ignore_index=True)

In [179]:
df

,category,wfh,location
0,"[Data Science, Machine Learning]",yes,"[Bangalore, Delhi]"
1,"[Data Science, Machine Learning]",yes,"[Bangalore, Delhi]"


In [12]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("notebooks"), '..')))
from src.web_scraper.scraper_functions import format_categories, format_locations

In [27]:
# from scraper_functions import format_categories, format_locations
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

class TooManyPages(Exception):
    def __init__(self, message="Please narrow search with filters, too many pages!"):
        self.message = message
        super().__init__(self.message)


def scraper(args_dict):

    base_url = "https://internshala.com/internships/"
    cat = args_dict["category"]
    wfh = args_dict["wfh"]
    loc = args_dict["location"]

    if wfh == "yes":
        base_url += "work-from-home-"
        wfh_status = True
        base_url += format_categories(cat, wfh_status)
    elif wfh == "no":
        wfh_status = False
        base_url += format_categories(cat, wfh_status)

    if len(loc) > 0:
        base_url += format_locations(loc)

    req = requests.get(base_url)
    soup = BeautifulSoup(req.content, "html.parser")
    total_pages = int(soup.find(id='total_pages').text.strip())

    if total_pages > 3:
        raise TooManyPages
    
    else:

        main_df = pd.DataFrame()

        for page in range(total_pages):
            pg_url = base_url
            pg_url += f"/page-{page+1}/"
            # print(pg_url)
            req_pg = requests.get(pg_url)
            soup = BeautifulSoup(req_pg.content, "html.parser")
            internships = soup.find_all(class_ = 'heading_4_5 profile')
            for internship in tqdm(internships):
                sub_info = {}
                sub_url = "https://internshala.com"
                sub_url += internship.find('a', href=True)["href"]
                req_sub_pg = requests.get(sub_url)
                sub_soup = BeautifulSoup(req_sub_pg.content, "html.parser")
                sub_info["title"] = sub_soup.find(class_ = 'profile_on_detail_page').text.strip()
                sub_info["company"] = sub_soup.find(class_ = 'heading_6 company_name').find('a').text.strip()
                sub_info["location"] = [i.text.strip() for i in sub_soup.find_all(class_ = 'location_link')]
                info = sub_soup.find(class_ = 'internship_other_details_container')
                other_details = info.find_all(class_ = 'item_body')
                sub_info["duration"] = other_details[1].text.strip()
                sub_info["stipend"] = other_details[2].text.strip()
                sub_info["apply_by"] = other_details[3].text.strip()
                sub_info["applicants"] = sub_soup.find(class_ = 'applications_message').text.strip()
                skills_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Skill(s) required')
                try:
                    skills_raw = skills_raw.findNext(class_ = 'round_tabs_container')
                    skills_raw.find_all(class_ = 'round_tabs')
                    sub_info["skills"] = [i.text.strip() for i in skills_raw.find_all(class_ = 'round_tabs')]
                except(AttributeError):
                    sub_info["skills"] = []
                try:
                    perks_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Perks')
                    perks_raw = perks_raw.findNext(class_ = 'round_tabs_container')
                    sub_info["perks"] = [i.text.strip() for i in perks_raw.find_all(class_ = 'round_tabs')]
                except(AttributeError):
                    sub_info["perks"] = []
                sub_info["openings"] = sub_soup.find_all(class_='text-container')[-1].text.strip()
                sub_info["url"] = sub_url
                sub_df = pd.DataFrame([sub_info])
                main_df = pd.concat([main_df, sub_df], ignore_index=True)

        return main_df.to_csv("raw_scraped.csv", index=False)

scraper({
    "category": ["Machine Learning"],
    "wfh": "yes",
    "location": []
})

100%|██████████| 13/13 [00:08<00:00,  1.60it/s]


In [17]:
args_dict = {
    "category": ["data science, machine learning"],
    "wfh": "yes",
    "location": []
}
scraper(args_dict)

100%|██████████| 2/2 [00:00<00:00,  2.15it/s]


In [26]:
df = pd.read_csv("raw_scraped.csv")
df

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
0,Teaching (Data Science),Ekeeda Private Limited,['Work From Home'],3 Months,10000-12000 /month,26 Jul' 22,54 applicants,[],"['Certificate', 'Flexible work hours']",10,https://internshala.com/internship/detail/teac...
1,Data Analytics,9 Realms Productions,['Work From Home'],2 Months,6500 /month,26 Jul' 22,60 applicants,"['Data Analytics', 'Data Science', 'MS-Excel']","['Certificate', 'Letter of recommendation', 'F...",3,https://internshala.com/internship/detail/data...
2,Teaching (Basics of Python & Machine Learning),SURE Trust,['Work From Home'],4 Months,Unpaid,25 Jul' 22,107 applicants,"['Machine Learning', 'Python']","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/teac...
3,Market Research Analysis,Strategy Here,['Work From Home'],6 Months,4000 /month,26 Jul' 22,44 applicants,"['Content Marketing', 'Market Research', 'Sale...","['Certificate', 'Flexible work hours', '5 days...",2,https://internshala.com/internship/detail/mark...
4,Course Development,Childo Education Research And Development Foun...,['Work From Home'],1 Month,Unpaid,23 Jul' 22,266 applicants,[],"['Certificate', 'Letter of recommendation', 'F...",5,https://internshala.com/internship/detail/cour...
5,Curriculum Development (AI/ML),Sustainable Living Lab (SL2),['Work From Home'],3 Months,15000-20000 /month,22 Jul' 22,624 applicants,"['Artifical Intelligence', 'Data Science', 'Ma...","['Certificate', 'Letter of recommendation', 'F...",3,https://internshala.com/internship/detail/curr...
6,Data Science,ML Concepts,['Work From Home'],6 Months,1000 /month + Incentives,22 Jul' 22,116 applicants,"['Artifical Intelligence', 'Data Analytics', '...","['Certificate', 'Letter of recommendation', 'F...",3,https://internshala.com/internship/detail/data...
7,Machine Learning,BsHappy Technologies,['Work From Home'],6 Months,3000 /month,21 Jul' 22,409 applicants,"['Artifical Intelligence', 'Data Analytics', '...","['Certificate', 'Letter of recommendation', 'F...",8,https://internshala.com/internship/detail/mach...
8,Data Science,MissionEd,['Work From Home'],2 Months,1000-4000 /month,21 Jul' 22,375 applicants,"['Deep Learning', 'Machine Learning', 'Python']","['Certificate', 'Letter of recommendation', 'F...",15,https://internshala.com/internship/detail/data...
9,Business Analytics,Urbano InfoTech Private Limited,['Work From Home'],6 Months,5000 /month,21 Jul' 22,309 applicants,"['Figma', 'MS-Excel', 'Software Testing']","['Certificate', 'Letter of recommendation', 'J...",2,https://internshala.com/internship/detail/busi...
